In [1]:
import pymongo
import pandas as pd
import re

In [2]:
# write id and password for your mongodb atlas account 
myclient = pymongo.MongoClient("mongodb+srv://<id>:<password>@cluster0.5xyb5sn.mongodb.net/?retryWrites=true&w=majority")
mydb = myclient["new"]

coll1 = mydb["us_stocks"]
coll2 = mydb["rating_history"]
stocks_df_dict = coll1.find({})
rating_history_dict = coll2.find({})

stocks_df = pd.DataFrame(stocks_df_dict)
stocks_df=stocks_df.drop('_id',axis=1)

company_rating_df = pd.DataFrame(rating_history_dict)
company_rating_df=company_rating_df.drop('_id',axis=1)


In [3]:
stocks_df.head()

,Symbol,Name,Sector,industry
0,AA,Alcoa Corporation,Basic Industries,Aluminum
1,AACG,ATA Creativity Global,Consumer Services,Other Consumer Services
2,AAL,"American Airlines Group, Inc.",Transportation,Air Freight/Delivery Services
3,AAMC,Altisource Asset Management Corp,Finance,Real Estate
4,AAME,Atlantic American Corporation,Finance,Life Insurance


In [4]:
company_rating_df.head()

,Rating Agency Name,Name,Rating Action Date,Rating
0,HR Ratings de Mexico S.A. de C.V.,Petroleos Mexicanos,NaN,HR A - (G)
1,HR Ratings de Mexico S.A. de C.V.,Petroleos Mexicanos,NaN,HR A - (G)
2,HR Ratings de Mexico S.A. de C.V.,Petroleos Mexicanos,NaN,HR A - (G)
3,HR Ratings de Mexico S.A. de C.V.,Petroleos Mexicanos,NaN,HR A - (G)
4,HR Ratings de Mexico S.A. de C.V.,Vasconia,NaN,HR A


In [5]:
company_rating_df['Rating Action Date'] = pd.to_datetime(company_rating_df['Rating Action Date'])

In [6]:
company_rating_df.isna().sum()

Rating Agency Name        0
Name                      0
Rating Action Date    82377
Rating                    0
dtype: int64

In [7]:
company_rating_df.dropna(inplace=True)
company_rating_df.drop_duplicates(inplace=True)


In [8]:
len(company_rating_df),len(stocks_df)

(83424, 7021)


### Standarize the ratings

We Need to Change some Ratings as per the standard Ratings Format

In [9]:
def standarizeRating(name):
    
    name = re.sub(r'Aaa','AAA', name)
    name = re.sub(r'Aa1', 'AA+', name)
    name = re.sub(r'Aa2', 'AA', name)
    name = re.sub(r'Aa3	', 'AA−', name)
    name = re.sub(r'A1', 'A+', name)    
    name = re.sub(r'A2','A', name)   
    name = re.sub(r'A3', 'A-', name)
    name = re.sub(r'Baa1','BBB+', name)
    name = re.sub(r'Baa2','BBB', name)
    name = re.sub(r'Baa3', 'BBB-', name)
    name = re.sub(r'Ba1','BB+', name)
    name = re.sub(r'Ba2', 'BB', name)
    name = re.sub(r'Ba3', 'BB−', name)
    name = re.sub(r'B1','B+', name)    
    name = re.sub(r'B2','B', name)   
    name = re.sub(r'B3','B−', name)
    name = re.sub(r'Caa1','CCC+', name)
    name = re.sub(r'Caa2','CCC', name)
    name = re.sub(r'Caa3', 'CCC−', name)
    return name

Apply transformation

In [10]:
company_rating_df['Rating'] = company_rating_df['Rating'].apply(str).apply(standarizeRating)

Some of the ratings contains A+ and A-

We remove + and - from ratings

In [11]:
company_rating_df['Rating'] = company_rating_df['Rating'].str.replace(r'+', '').str.replace(r'-', '')

We observed that ratings column contain non traditional ratings, so we remove non-traditional ratings from the ratings column

In [12]:
traditional_rating_set = {'A', 'AA', 'AAA', 
              'B', 'BB', 'BBB', 
              'C', 'CC', 'CCC', 
              'D'}

company_rating_df = company_rating_df[company_rating_df['Rating'].isin(traditional_rating_set)]

We observed that some of the data contails duplicate data. To reduce data redundency, we will drop duplicate records

In [13]:
company_rating_df = company_rating_df.drop_duplicates(subset = company_rating_df.columns)

Cheking Ratings wise Records length

In [14]:
company_rating_df.groupby('Rating').count()

,Rating Agency Name,Name,Rating Action Date
Rating,,,
A,10810,10810,10810
AA,2500,2500,2500
AAA,269,269,269
B,8514,8514,8514
BB,8256,8256,8256
BBB,15691,15691,15691
C,255,255,255
CC,130,130,130
CCC,3165,3165,3165


We want to merge the datasets stocks_df and company_rating_df

We need to Clean company names because Company Names Contains unwated characters

The best method is to remove all possible characters that will cause mismatch

We will Define a function that will remove all characters and names we don't want

In [15]:
def company_name_clean(name):
    
    name = name.lower()
    name = re.sub(r'\.', '', name)
    name = re.sub(r',', '', name)
    name = re.sub(r'^a-z', '', name)
    name = re.sub(r'corporation', '', name)
    name = re.sub(r' corp', '', name)    
    name = re.sub(r' co', '', name)   
    name = re.sub(r'inc', '', name)
    name = re.sub(r'limited', '', name)
    name = re.sub(r'ltd', '', name)
    name = re.sub(r'holdings', '', name)
    name = re.sub(r' holding', '', name)    
    name = re.sub(r'plc', '', name)
    name = re.sub(r'group', '', name)
    name = re.sub(r' ag', '', name)
    name = re.sub(r' sa', '', name)
    name = re.sub(r' pty', '', name)
    name = re.sub(r' international', '', name)
    name = re.sub(r' incorporated', '', name)
    name = re.sub(r' spa', '', name)
    name = re.sub(r' se', '', name)
    name = re.sub(r' lp', '', name)    
    name = re.sub(r' (The)', '', name)    
    name = re.sub(r'The', '', name)    
    name = re.sub(r'LLC', '', name)   
    name = re.sub(r'n.v', '', name)   
    name = name.strip()
    
    return name



We will Apply the function for both datasets

In [16]:
#stocks_df
stocks_df.loc[:, 'Clean_Name'] = stocks_df['Name']
stocks_df.loc[:, 'Clean_Name'] = stocks_df['Name'].apply(company_name_clean)
#company_rating_df
company_rating_df.loc[:, 'Clean_Name'] = company_rating_df['Name']
company_rating_df.loc[:, 'Clean_Name'] = company_rating_df['Clean_Name'].apply(str).apply(company_name_clean)

Merge both the Datasets

In [17]:
company_rating_df = pd.merge(company_rating_df, stocks_df ,on='Clean_Name')

In [18]:
company_rating_df.columns

Index(['Rating Agency Name', 'Name_x', 'Rating Action Date', 'Rating',
       'Clean_Name', 'Symbol', 'Name_y', 'Sector', 'industry'],
      dtype='object')

Changing the columns name from company_rating_df and droping unwanted columns

In [19]:
company_rating_df = company_rating_df.rename(columns = {'Name_x': 'Name'}).drop('Name_y', axis = 1)

Now Again Visualise the data

In [20]:
company_rating_df.groupby('Rating').count()

,Rating Agency Name,Name,Rating Action Date,Clean_Name,Symbol,Sector,industry
Rating,,,,,,,
A,2482,2482,2482,2482,2482,2482,2482
AA,656,656,656,656,656,656,656
AAA,74,74,74,74,74,74,74
B,1114,1114,1114,1114,1114,1114,1114
BB,2027,2027,2027,2027,2027,2027,2027
BBB,3825,3825,3825,3825,3825,3825,3825
C,21,21,21,21,21,21,21
CC,49,49,49,49,49,49,49
CCC,370,370,370,370,370,370,370


In [21]:
company_rating_df.drop_duplicates(subset = ['Rating Action Date', 'Clean_Name'])

,Rating Agency Name,Name,Rating Action Date,Rating,Clean_Name,Symbol,Sector,industry
0,Standard & Poor's Ratings Services,Southwest Gas Corp.,2021-12-28,AA,southwest gas,SWX,Public Utilities,Oil & Gas Production
1,Moody's Investors Service,Southwest Gas Corporation,2021-09-27,A,southwest gas,SWX,Public Utilities,Oil & Gas Production
2,Standard & Poor's Ratings Services,Southwest Gas Corp.,2021-01-21,AA,southwest gas,SWX,Public Utilities,Oil & Gas Production
3,Standard & Poor's Ratings Services,Southwest Gas Corp.,2020-04-17,AA,southwest gas,SWX,Public Utilities,Oil & Gas Production
4,Moody's Investors Service,Southwest Gas Corporation,2019-01-31,A,southwest gas,SWX,Public Utilities,Oil & Gas Production
...,...,...,...,...,...,...,...,...
10622,Standard & Poor's Ratings Services,IRSA Inversiones y Representaciones S.A.,2015-12-17,B,irsa inversiones y representaciones,IRS,Consumer Services,Homebuilding
10623,Moody's Investors Service,Cooper-Standard Holdings Inc.,2018-03-19,CCC,cooper-standard,CPS,Capital Goods,Auto Parts:O.E.M.
10624,Moody's Investors Service,Mallinckrodt plc,2018-03-18,BB,mallkrodt,MNK,Health Care,Major Pharmaceuticals
10625,Fitch Ratings,B Communications Ltd.,2019-02-18,BB,bmmunications,BCOM,Consumer Services,Telecommunications Equipment


### We will add financial Ratings columns from API Financial Modeling Prep

Create function to get data

In [22]:
def get_jsonparsed_data(url):
    
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

We can get 5 years of financial Ratings from API Financial Modeling Prep using the company symbol

we will make a set of symbols from our "Symbol" column of "company_rating_df"

In [23]:
Symbol_list = list(set(company_rating_df.Symbol))

# Remove Unwated characters
Symbol_list = [re.sub(r'\^.+', '', item) for item in Symbol_list]
len(Symbol_list)

935

We can get 5 years of financial Ratios from API Financial Modeling Prep using the company symbol

As we are using a free account we can make 250 requests / day.

So, We create financial Ratios from API and store them in Local files and use them for further requirements

In [24]:
financial_ratios_df = pd.DataFrame()
financial_ratios_df.head()

""


In [25]:
import json
from urllib.request import urlopen

# Get every company from the list the financial information
for Symbol in Symbol_list[880:]: 
    if Symbol.isalpha() == True:
       
        ratios = pd.DataFrame.from_dict(
                get_jsonparsed_data("https://financialmodelingprep.com/api/v3/ratios/" 
                                    + Symbol + 
                                    "?limit=40&apikey=ea9f74285ac2407d723cf1356da4d9be"))
        
        frames = [financial_ratios_df, ratios]
       
        financial_ratios_df = pd.concat(frames)

In [26]:
#financial_ratios_df.to_csv(r'/home/talentum/spark/Untitled Folder/corporateCreditRating_project/financial_ratios_df.csv')
#financial_ratios_df_all_from_api = financial_ratios_df

In [27]:
########## Merging Daywise Data
financial_ratios_df = pd.read_csv(r'/home/talentum/spark/Untitled Folder/corporateCreditRating_project/financial_ratios_df.csv')
financial_ratios_df_all_from_api = financial_ratios_df.drop(['Unnamed: 0'] , axis=1)


Financial Ratios from API Financial Modeling Prep returns every available information from the past 5 year wise

we want exclusively the data from the year of the rating or the year before If the rating was done before june we will take the indicators from the year before If it was after june we will take from the following year

In [28]:
company_rating_df = company_rating_df.rename(columns = {'Rating Action Date':'date'})
company_rating_df['date'] = company_rating_df['date'].astype('datetime64[ns]')
company_rating_df['month_rating'] = company_rating_df['date'].dt.month
company_rating_df['year_change'] = [1 if x < 7 else 0 for x in company_rating_df['month_rating']]
company_rating_df['year'] = company_rating_df['date'].dt.year
company_rating_df['Year'] = company_rating_df['year'] - company_rating_df['year_change']



financial_ratios_df_all_from_api = financial_ratios_df_all_from_api.rename(columns = {'symbol':'Symbol'})
financial_ratios_df_all_from_api['date'] = financial_ratios_df_all_from_api['date'].astype('datetime64[ns]')
financial_ratios_df_all_from_api['Year'] = financial_ratios_df_all_from_api['date'].dt.year


In [29]:
company_rating_df.head(5)

,Rating Agency Name,Name,date,Rating,Clean_Name,Symbol,Sector,industry,month_rating,year_change,year,Year
0,Standard & Poor's Ratings Services,Southwest Gas Corp.,2021-12-28,AA,southwest gas,SWX,Public Utilities,Oil & Gas Production,12,0,2021,2021
1,Moody's Investors Service,Southwest Gas Corporation,2021-09-27,A,southwest gas,SWX,Public Utilities,Oil & Gas Production,9,0,2021,2021
2,Standard & Poor's Ratings Services,Southwest Gas Corp.,2021-01-21,AA,southwest gas,SWX,Public Utilities,Oil & Gas Production,1,1,2021,2020
3,Standard & Poor's Ratings Services,Southwest Gas Corp.,2020-04-17,AA,southwest gas,SWX,Public Utilities,Oil & Gas Production,4,1,2020,2019
4,Moody's Investors Service,Southwest Gas Corporation,2019-01-31,A,southwest gas,SWX,Public Utilities,Oil & Gas Production,1,1,2019,2018


In [30]:
financial_ratios_df_all_from_api.head(3)

,Symbol,date,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,...,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,Year
0,MANT,2021-12-31,FY,1.583978,1.462110,0.141490,71.196949,0.0,71.196949,28.390353,...,21.012502,18.308406,13.569574,13.569574,1.653352,1.127298,0.021473,12.202877,1.718417,2021
1,MANT,2020-12-31,FY,1.423052,1.329601,0.118095,61.247604,0.0,61.247604,24.294753,...,29.284529,20.650731,14.276036,14.276036,5.961493,1.401559,0.014624,15.808825,2.234796,2020
2,MANT,2019-12-31,FY,1.515876,1.432844,0.029515,69.134174,0.0,69.134174,28.147313,...,27.585185,19.281898,14.189664,14.189664,0.745158,1.413540,0.013752,16.288438,2.107600,2019


Merging Both Dataset on Symbol and Year

In [31]:
company_rating_df = pd.merge(company_rating_df, financial_ratios_df_all_from_api, on=['Symbol', 'Year']) 

In [32]:
company_rating_df.shape

(8241, 68)

This DataFrame have many null values and duplicate valies. we will set a threshold tolarance for null values in a column, if a column have more then threshold null values we will drop that column

In [33]:
a = company_rating_df.count()
the_list = a[a<8000].reset_index()['index']

company_rating_df_NA = company_rating_df[company_rating_df.columns.difference(the_list)]
company_rating_df_NA = company_rating_df_NA.dropna()
company_rating_df_NA = company_rating_df_NA.drop_duplicates(['Clean_Name', 'date_y'])

In [34]:
company_rating_df_NA.shape , company_rating_df.shape

((2034, 46), (8241, 68))

In [35]:
company_rating_df_NA = company_rating_df_NA.drop(['Name', 'date_y' , 'date_x', 'ebtPerEbit',
                                    'month_rating', 'netIncomePerEBT', 'year',
                                    'year_change', 'industry',
                                    'priceCashFlowRatio', 'priceEarningsRatio',	
                                    'priceSalesRatio',
                                    'priceToFreeCashFlowsRatio',	
                                    'priceToOperatingCashFlowsRatio',	
                                    'priceToSalesRatio','period' ], axis=1)

In [36]:
company_rating_df_NA.shape

(2034, 30)

Renaming the columns columns

In [37]:
company_rating_df_NA = company_rating_df_NA.rename({'Clean_Name' :'Name', 'date_x':'Date'})

In [38]:
company_rating_df_NA.head()

,Clean_Name,Rating,Rating Agency Name,Sector,Symbol,Year,assetTurnover,cashFlowToDebtRatio,cashPerShare,cashRatio,...,netProfitMargin,operatingCashFlowPerShare,operatingCashFlowSalesRatio,operatingProfitMargin,payoutRatio,pretaxProfitMargin,priceEarningsToGrowthRatio,quickRatio,returnOnAssets,returnOnCapitalEmployed
0,southwest gas,AA,Standard & Poor's Ratings Services,Public Utilities,SWX,2021,0.288318,0.017618,3.976874,0.071560,...,0.054553,1.989053,0.030263,0.100406,0.688429,0.067071,-1.398241,0.331467,0.015729,0.038282
2,southwest gas,AA,Standard & Poor's Ratings Services,Public Utilities,SWX,2020,2.712450,0.217417,1.488482,0.091398,...,0.072444,11.180399,0.189786,0.128227,0.525154,0.092376,2.635970,0.663976,0.196502,1.390409
3,southwest gas,AA,Standard & Poor's Ratings Services,Public Utilities,SWX,2019,0.381873,0.187055,0.913245,0.045875,...,0.068571,9.224297,0.160380,0.119173,0.542812,0.087397,2.868255,0.587564,0.026185,0.052440
7,southwest gas,A,Moody's Investors Service,Public Utilities,SWX,2018,0.391427,0.230708,1.727291,0.090941,...,0.063290,10.701471,0.183630,0.124108,0.549932,0.084491,-2.389376,0.629780,0.024774,0.055683
11,southwest gas,A,Fitch Ratings,Public Utilities,SWX,2017,0.408652,0.181491,0.905547,0.053466,...,0.076052,7.679876,0.145149,0.127001,0.475286,0.101629,0.695363,0.584837,0.031079,0.059710


In [39]:
company_rating_dict = company_rating_df_NA.to_dict('record')

import pymongo

# write mongodb your atlas id and password
myclient = pymongo.MongoClient("mongodb+srv://<id>:<password>@cluster0.5xyb5sn.mongodb.net/?retryWrites=true&w=majority")
mydb = myclient["new"]

coll1 = mydb["corparate_credit_rating_datset"]

#coll1.insert_many(company_rating_dict)


/home/talentum/.local/lib/python3.6/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [40]:
company_rating_df_NA.shape

(2034, 30)